# My Server の Container イメージをリストアする
<HR>

My Server の Container イメージファイルから、Container イメージを各Notebook Container ホストに配布し、配布したイメージで My Server が起動するように設定します。

# Container イメージファイルを格納するディレクトリの用意

配布元の Container イメージファイルを格納するディレクトリです。

`BACKUP_WORK_DIR`は、`$HOME`からの相対パスです。

In [ ]:
BACKUP_WORK_DIR = 'images'

import os
import os.path
backup_dir = os.path.expanduser(os.path.join('~', BACKUP_WORK_DIR))
if not os.path.exists(backup_dir):
    os.makedirs(backup_dir)

# Container イメージファイルを転送

* リストアする Container イメージバックアップファイルを、ディレクトリ `$HOME/images` に転送して下さい。
* イメージファイルは `docker save`コマンドで作られる tar 形式のファイルです。
*  イメージファイルを、`16_SaveNotebookContainerImage.ipynb` 作成した場合は、すでに`$HOME/images`に保存されていますので、先へ進んでください


# Container イメージバックアップファイル名を指定

* 次のセルで、`ARCHIVE_FILE_NAME` にリストアする Container イメージのバックアップファイル名を指定して下さい。
```
ARCHIVE_FILE_NAME="server-container-170513-065930.tar"
```
* バックアップファイルは、上のセルで作成した BACKUP_WORK_DIR に格納されているものとします。

In [ ]:
!cd ~/{BACKUP_WORK_DIR} && ls -l *.tar

In [ ]:
ARCHIVE_FILE_NAME="server-container-20190322-020641.tar"

In [ ]:
import os.path
imagefilepath = os.path.expanduser(os.path.join('~', BACKUP_WORK_DIR, ARCHIVE_FILE_NAME))
assert os.path.exists(imagefilepath)
!ls -l ~/{BACKUP_WORK_DIR}/{ARCHIVE_FILE_NAME}

# Containerイメージの配布

CoursewareHubの各ノードにイメージを配布します。

In [ ]:
import os
target_hub = ['-i', os.path.expanduser('~/ansible/inventory'), 'ch-master']

!ansible -m ping {' '.join(target_hub)}

In [ ]:
target_hub_all = ['-i', os.path.expanduser('~/ansible/inventory'), 'ch-all']

!ansible -m ping {' '.join(target_hub_all)}

**上記セルが動作しない場合**

```
The authenticity of host 'xxx.xxx.205.128 (xxx.xxx.205.128)' can't be established.
ECDSA key fingerprint is SHA256:qjPDx7y/926gHJL9+SgMGKpicRORzffk1/xiUyIP00w.
Are you sure you want to continue connecting (yes/no)?
```
（IPアドレスと、fingerprintは例です）

となり実行中のまま状態変化しなくなる場合は、JupyterのTerminalから、

```
$ ssh xxx.xxx.205.128
```

を実行し、ECDSA key fingerprintが `SHA256:qjPDx7y/926gHJL9+SgMGKpicRORzffk1/xiUyIP00w` であることを確認してyesを実行し、上記のセルを停止の上再実行してください。

## 現行版のイメージの情報を控える

新しいイメージに問題があった場合に、現行版のイメージを復元するための情報を控えておきます。

In [ ]:
def get_container(name):
    import subprocess
    try:
        sid = subprocess.check_output(['ansible', '-b', '-a', 'docker service ps {} -q'.format(name)] + target_hub)
        sid = sid.decode('utf-8').split('\n')[1].strip()
        cinfo = subprocess.check_output(
            ['ansible', '-b', '-a', 
             'docker inspect --format "{% raw %} {{.NodeID}} {{.Status.ContainerStatus.ContainerID}} {% endraw %}" ' + sid
            ] + target_hub)
        nodeid, cid = cinfo.decode('utf-8').split('\n')[1].strip().split()
        nodeip = subprocess.check_output(
            ['ansible', '-b', '-a', 
             'docker node inspect --format "{% raw %} {{.Status.Addr}} {% endraw %}" ' + nodeid
            ] + target_hub)
        nodeip = nodeip.decode('utf-8').split('\n')[1].split()[0]
        return (nodeip, cid)
    except subprocess.CalledProcessError as e:
        print(e.output.decode('utf-8'))
        raise

def get_container_env(service_name, env_name):
    c = get_container(service_name)

    import subprocess
    import json
    target = ['-i', os.path.expanduser('~/ansible/inventory'), c[0]]
    env = subprocess.check_output(
        ['ansible', '-b', '-a', 
         'docker inspect --format "{% raw %} {{json .Config.Env}} {% endraw %}" ' + c[1]
        ] + target)
    env = json.loads(env.decode('utf-8').split('\n')[1])
    env = dict([tuple(x.split('=')) for x in env])
    return env[env_name]

image_tag = get_container_env('jupyterhub', 'CONTAINER_IMAGE')
image_tag

In [ ]:
import subprocess
org_image_id = subprocess.check_output(
    ['ansible', '-b', '-a', 
     'docker inspect --format "{% raw %} {{.Id}} {% endraw %}" ' + image_tag
    ] + target_hub)
org_image_id = org_image_id.decode('utf-8').split('\n')[1].strip()
org_image_id

万が一、起動不可能になったとき管理者が復旧作業ができるように、現行版のイメージにタグをつけておきます。

In [ ]:
from datetime import datetime

image_tag_body = image_tag.rsplit(':', 1)[0]
now = datetime.now()

failback_latest_tag = '{}:fallback-latest'.format(image_tag_body)
failback_date_tag = '{}:fallback-{}'.format(image_tag_body, now.strftime('%Y%m%d-%H%M%S'))

print('Setting image tag {} => {}'.format(org_image_id, failback_date_tag) )
subprocess.check_output(['ansible', '-b', '-a', 
                               'docker tag {} {}'.format(org_image_id, failback_date_tag)]
                              + target_hub_all).decode('utf-8')
print('Setting image tag {} => {}'.format(org_image_id, failback_latest_tag) )
subprocess.check_output(['ansible', '-b', '-a', 
                               'docker tag {} {}'.format(org_image_id, failback_latest_tag)]
                              + target_hub_all).decode('utf-8')

## イメージの配布

イメージの配布を実行します。

In [ ]:
import subprocess
import re

print('Loading image from the file: {}'.format(imagefilepath))
loaded_image = subprocess.check_output(['ansible', '-b', '-a', 
                                        'docker load -q -i /jupyter/users/{}/{}/{}'.format(
                                            os.environ['JUPYTERHUB_USER'],
                                            BACKUP_WORK_DIR,
                                            ARCHIVE_FILE_NAME,
                                        )] + target_hub_all)
loaded_image = loaded_image.decode('utf-8').split('\n')
loaded_images = set()
for l in loaded_image:
    m = re.match(r'^Loaded image:\s*(.+)$', l)
    if m:
        loaded_images.add(m.group(1))
assert len(loaded_images) == 1
loaded_image = loaded_images.pop()
print('Loaded image: {}'.format(loaded_image))

print('Set image tag {} => {}'.format(loaded_image, image_tag) )
subprocess.check_output(['ansible', '-b', '-a', 
                         'docker tag {} {}'.format(loaded_image, image_tag)]
                        + target_hub_all)

print('Completed')

## テスト

1. 以下のURLにアクセスして、Adminのコントロールパネルを開いてください。
  * [/hub/admin](/hub/admin)
1. 現在使用しているユーザー**以外**の**テスト用のユーザー**で、「stop server」、「start server」でJupyter Notebokの再起動を行ってください。
  * 起動可能であると確認できるまでは、自分のユーザーでは試さないでください。
  * テスト用のユーザーがない場合は、あらかじめ作成しておく必要があります。
1. 「access server」ボタンで、テスト用ユーザーのJupyter Notebookサーバーにアクセスし、動作を確認してください。
1. 新しいイメージは、各ユーザーのMy Serverを再起動するまでは反映されません。

## 復元

このセクションの手順は、上のテストで、配布したイメージが動作しなかった場合に、元のイメージ復元するためのものです。

成功した場合は、実行の必要はありません。

In [ ]:
# Run All で以下のセルが実行されるのを防止します
assert False

In [ ]:
print('Setting image tag {} => {}'.format(org_image_id, image_tag) )
subprocess.check_output(['ansible', '-b', '-a', 
                               'docker tag {} {}'.format(org_image_id, image_tag)]
                              + target_hub_all).decode('utf-8')
print('Completed')